# Playing around with Flux1

In [ ]:
%pip install "gguf[gui]" diffusers transformers accelerate bitsandbytes sentencepiece

## Select Device

In [1]:
# choose device: prefer CUDA, then MPS, then CPU
import torch

try:
    cuda_available = torch.cuda.is_available()
except Exception:
    cuda_available = False

mps_available = False
try:
    mps_backend = getattr(torch.backends, "mps", None)
    mps_available = mps_backend is not None and mps_backend.is_available()
except Exception:
    mps_available = False

if cuda_available:
    device = torch.device("cuda")
elif mps_available:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Selected device: {device}")

Selected device: cuda


## FIXME: Below crashes Jupyter due to OOM?

In [ ]:
from diffusers import FluxPipeline, FluxTransformer2DModel, GGUFQuantizationConfig
import torch
import gc

#torch.mps.set_per_process_memory_fraction(0.0)

def flush():
    gc.collect()
    #torch.mps.empty_cache()
    gc.collect()
    #torch.mps.empty_cache()

prompt = "a moonim dressed as a knight, riding a horse towards a medieval castle"

ckpt_id = "black-forest-labs/FLUX.1-dev"

pipeline = FluxPipeline.from_pretrained(
    ckpt_id,
    transformer=None,
    vae=None,
    dtype=torch.bfloat16,
).to(device)

with torch.no_grad():
    print("Encoding prompts.")
    prompt_embeds, pooled_prompt_embeds, text_ids = pipeline.encode_prompt(
        prompt=prompt, prompt_2=prompt, max_sequence_length=256
    )


del pipeline
flush()

#ckpt_path = "https://huggingface.co/city96/FLUX.1-dev-gguf/blob/main/flux1-dev-Q8_0.gguf"
ckpt_path = "/home/sral/Downloads/flux1-dev-Q8_0.gguf"

transformer = FluxTransformer2DModel.from_single_file(
    ckpt_path,
    quantization_config=GGUFQuantizationConfig(compute_dtype=torch.bfloat16),
    torch_dtype=torch.bfloat16,
)

pipeline = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    text_encoder=None,
    text_encoder_2=None,
    tokenizer=None,
    tokenizer_2=None,
    transformer=transformer,
    dtype=torch.bfloat16,
).to(device)

print("Running denoising.")
height, width = 1024, 1024
# No need to wrap it up under `torch.no_grad()` as pipeline call method
# is already wrapped under that.
images = pipeline(
    prompt_embeds=prompt_embeds,
    pooled_prompt_embeds=pooled_prompt_embeds,
    num_inference_steps=15,
    guidance_scale=5.0,
    height=height,
    width=width,
    generator=torch.Generator(device).manual_seed(42)
).images[0]

images.save("compile_image.png")